In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import tensorflow as tf
import statistics as stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.regularizers import l2
from numpy.random import seed
from keras.wrappers.scikit_learn import KerasRegressor
from keras.regularizers import l2
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [2]:
# ensure repeatability
np.random.seed(23)

In [3]:
data = pd.read_csv("ML_data.csv", sep="|")
pd.set_option("display.max_columns", None)

In [4]:
data.head()

,Type,AB,Site,Impurity,dH (A-rich),dH (B-rich),(+2/+1),(+1/0),(0/-1),(-1/-2),Z_A,Z_B,PBE_delta_H,PBE_latt_const,PBE_gap,Eps_elec,Eps_ion,CM1,CM2,CM3,CM4,CM5,CM6,CM7,CM8,Ion_rad,BP,MP,Density,At_wt,ICSD_vol,Cov_rad,Ion_Energy,At_rad_1,Elec_Aff,At_rad_2,At_vol,Mend_num,Ion_pot_1,Ion_pot_2,Thermal_expn,Sp_heat_cap,Therm_cond,Elec_cond,Heat_fusion,Heat_vap,Electronegativity,At_num,Period,Group,Valence,Ox_state
0,IV-IV,SiC,M_i_A,In,15.963,15.963,3.891,4.660,5.109,5.629,14,6,-0.42,4.25,1.27,18.06,2.27,373.01,373.01,373.01,373.01,138.44,138.44,138.44,79.93,0.80,2350.0,429.78,7.31,114.82,26.1,1.44,558.3,1.66,29.0,2.00,15.70,75,5.79,18.87,32.1,0.23,81.60,3.4,3.26,226.34,1.78,49,5,13,3,3
1,IV-IV,SiC,M_i_B,La,19.972,19.554,3.863,4.539,4.991,5.556,14,6,-0.42,4.25,1.27,18.06,2.27,375.78,375.78,375.78,216.96,185.96,185.96,185.96,185.96,1.16,3737.0,1191.00,6.15,138.91,37.4,1.25,538.1,1.88,50.0,2.74,20.73,13,5.58,11.06,5.2,0.19,13.50,1.9,11.30,399.57,1.10,57,6,3,3,3
2,IV-IV,SiC,M_i_neut,Zn,7.157,7.157,3.206,4.244,4.734,5.363,14,6,-0.42,4.25,1.27,18.06,2.27,152.25,238.53,238.53,238.53,102.23,102.23,102.23,65.25,0.74,1180.0,692.73,7.13,65.39,15.1,1.25,906.4,1.33,0.0,1.53,9.20,69,9.39,17.96,30.2,0.39,116.00,16.9,7.38,115.30,1.65,30,4,12,2,2
3,IV-IV,SiC,M_i_B,Al,6.951,7.110,3.503,4.242,4.650,5.319,14,6,-0.42,4.25,1.27,18.06,2.27,85.70,85.70,85.70,49.48,42.41,42.41,42.41,42.41,0.54,2740.0,933.50,2.70,26.98,16.6,1.18,577.6,1.43,45.0,1.62,10.00,73,5.99,18.83,23.1,0.90,237.00,37.7,10.70,290.80,1.61,13,3,13,3,3
4,IV-IV,SiC,M_B,I,8.783,9.201,3.614,4.221,5.058,5.608,14,6,-0.42,4.25,1.27,18.06,2.27,403.46,403.46,403.46,403.46,6874.36,105.89,105.89,105.89,2.20,457.5,386.70,4.93,126.90,42.5,1.33,1008.4,1.33,295.3,1.32,25.74,96,10.45,19.13,87.0,0.14,0.45,0.0,7.76,20.90,2.66,53,5,17,7,1


In [5]:
############ DATA PRE-PROCESSING ############ 

x_data = data.iloc[:, 10:53]
y_data = data.iloc[:, 6:10]

# separate categorical and continuous data 
categorical=pd.DataFrame()
continuous=pd.DataFrame()

for index in x_data.columns:
    if(x_data[index].dtypes == "int"):
        categorical[index]=x_data[index]
    elif(x_data[index].dtypes == "float"):
        continuous[index]=x_data[index]
    else:
        pass

# one hot encode categorical data
onehotencoder = OneHotEncoder()
categorical = onehotencoder.fit_transform(categorical).toarray()

# standardize continuous data
scaler = StandardScaler()
continuous = scaler.fit_transform(continuous)

# re-combine categorical and continuous data
x = np.concatenate((continuous, categorical), axis=1)

# extract y data and standardize (DFT predicted / output)
y = scaler.fit_transform(y_data)

# split training and testing data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=23)

# input and output dimension
in_dim = x.shape[1]
out_dim = y.shape[1]

In [6]:
# construct model

def construct_model(learning_rate=5e-4, reg=0.05, d_o=0.01):
    
    # Create and add layers to model
    model = Sequential()
    model.add(Dense(out_dim*308, input_dim=in_dim, activation='relu',
              kernel_regularizer=l2(reg)))
    model.add(Dropout(d_o))
    model.add(Dense(out_dim*308, activation='relu', ))
    model.add(Dense(out_dim*154, activation='relu'))
    model.add(Dropout(d_o))
    model.add(Dense(out_dim*154, activation='relu'))
    model.add(Dense(out_dim))

    # configure optimizer & compile model
    opt = Adam(lr=learning_rate, decay=0)   ### need editing later (learning_rate, adam_decay)
    model.compile(loss="mse", optimizer=opt)

    #### summarize model
    # print(model.summary())
    
    return model

In [7]:
# parameter grid
epochs = [50]
batch_size = [10]
learning_rate = [2.5e-3] #, 2.3e-3, 2.1e-3]
reg = [0.01, 0.05]
d_o = [.01]

hyperparameters = dict(epochs=epochs,
                       batch_size=batch_size,
                       learning_rate=learning_rate,
                       reg=reg,
                       d_o=d_o
                      )

neural_network = KerasRegressor(build_fn=construct_model, verbose=0)

grid = GridSearchCV(estimator=neural_network, cv=2, param_grid=hyperparameters, n_jobs=-1,
                    verbose=18)
grid_result = grid.fit(x, y)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:  7.7min remaining:  7.7min
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  7.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  7.9min finished


In [8]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -1.626261 using {'batch_size': 10, 'd_o': 0.01, 'epochs': 50, 'learning_rate': 0.0025, 'reg': 0.01}
-1.626261 (0.429315) with: {'batch_size': 10, 'd_o': 0.01, 'epochs': 50, 'learning_rate': 0.0025, 'reg': 0.01}
-1.646409 (0.373753) with: {'batch_size': 10, 'd_o': 0.01, 'epochs': 50, 'learning_rate': 0.0025, 'reg': 0.05}


In [9]:
from datetime import datetime
import pytz

tz_NY = pytz.timezone('America/Los_Angeles') 
datetime_NY = datetime.now(tz_NY)
print("Finished Time:", datetime_NY.strftime("%H:%M:%S"))

Finished Time: 14:28:00
